Information about data:

                     ->We have the amazon reviews dataset from kaggle
                     
                     ->Reviews are given for the product
                     
                     ->The features of the data were:
                     
                           Id
                           
                           ProductId- unique identifier for the product
                           
                           UserId- unqiue identifier for the user
                           
                           ProfileName
                           
                           HelpfullnessNumerator- number of users who found the review helpful
                           
                           HelpfulnessDenominator- number of users who indicated whether they 
                                                   found the review  helpful or not
                           
                           Score-rating between 1 and 5
                           
                           Time-timestamp for the review
                           
                           Summary- brief summary of the review
                           
                           Text- text of the review
                       
                     -> Based on the score of the review  we classify them into positive and 
                        negative
                        
                     
                     Number of reviews: 568,454

                     
                    
                     
                  

Objective:

    -> Clean the data and to get the complete vocabulary
    
    -> To get frequencies of each word in the vocabulary
    
    -> Convert the text into the values of the word frequencies
    
    -> To try different layers of LSTM, with different number of LSTM unit cells in each layer

Loading the required libraries to process

In [60]:
import sqlite3
import nltk
import string
from sklearn.metrics import accuracy_score
from nltk.stem.porter import PorterStemmer

Loading the data

In [2]:
con = sqlite3.connect("database.sqlite")
data = pd.read_sql_query("SELECT * FROM Reviews",con)

Analysis about the given data:
    
    ->The shape of data
    
    ->Number of dimension of the data
    
    ->Number of attributes and their names
    
    ->Sample subset of data

In [96]:
print(data.shape)
print(data.ndim)
print(data.columns)
print(data.head(5))

(568454, 10)
2
Index(['Id', 'ProductId', 'UserId', 'ProfileName', 'HelpfulnessNumerator',
       'HelpfulnessDenominator', 'Score', 'Time', 'Summary', 'Text'],
      dtype='object')
   Id   ProductId          UserId                      ProfileName  \
0   1  B001E4KFG0  A3SGXH7AUHU8GW                       delmartian   
1   2  B00813GRG4  A1D87F6ZCVE5NK                           dll pa   
2   3  B000LQOCH0   ABXLMWJIXXAIN  Natalia Corres "Natalia Corres"   
3   4  B000UA0QIQ  A395BORC6FGVXV                             Karl   
4   5  B006K2ZZ7K  A1UQRSCLF8GW1T    Michael D. Bigham "M. Wassir"   

   HelpfulnessNumerator  HelpfulnessDenominator     Score        Time  \
0                     1                       1  positive  1303862400   
1                     0                       0  negative  1346976000   
2                     1                       1  positive  1219017600   
3                     3                       3  negative  1307923200   
4                     0         

NOTE:
    
    -> We have the reviews in the form of text and ratings are in the range of 1 to 5
    
    -> Converting the rating score into positive, negative based on their score
    
    -> Considering the reviews with rating score greater than 3 as positive and less than or equal to three as negative

In [97]:
def reviews(x):
    if x>3:
        return 'positive'
    return 'negative'

In [99]:
data.head(5)

,Id,ProductId,UserId,ProfileName,HelpfulnessNumerator,HelpfulnessDenominator,Score,Time,Summary,Text
0,1,B001E4KFG0,A3SGXH7AUHU8GW,delmartian,1,1,positive,1303862400,Good Quality Dog Food,I have bought several of the Vitality canned d...
1,2,B00813GRG4,A1D87F6ZCVE5NK,dll pa,0,0,negative,1346976000,Not as Advertised,Product arrived labeled as Jumbo Salted Peanut...
2,3,B000LQOCH0,ABXLMWJIXXAIN,"Natalia Corres ""Natalia Corres""",1,1,positive,1219017600,"""Delight"" says it all",This is a confection that has been around a fe...
3,4,B000UA0QIQ,A395BORC6FGVXV,Karl,3,3,negative,1307923200,Cough Medicine,If you are looking for the secret ingredient i...
4,5,B006K2ZZ7K,A1UQRSCLF8GW1T,"Michael D. Bigham ""M. Wassir""",0,0,positive,1350777600,Great taffy,Great taffy at a great price. There was a wid...


Exploratory data analysis

Deduplication: Removing duplicates

In [100]:
sorteddata = data.sort_values('ProductId',axis=0,inplace=False,ascending=True,kind='quicksort',na_position='last')

In [101]:
sorteddata.head(6)

,Id,ProductId,UserId,ProfileName,HelpfulnessNumerator,HelpfulnessDenominator,Score,Time,Summary,Text
150528,150529,0006641040,A25ACLV5KPB4W,"Matt Hetling ""Matt""",0,1,positive,1108425600,"Nice cadence, catchy rhymes",In June<br />I saw a charming group<br />of ro...
150506,150507,0006641040,A1S4A3IQ2MU7V4,"sally sue ""sally sue""",1,1,positive,1191456000,chicken soup with rice months,This is a fun way for children to learn their ...
150505,150506,0006641040,A2IW4PEEKO2R0U,Tracy,1,1,positive,1194739200,"Love the book, miss the hard cover version","I grew up reading these Sendak books, and watc..."
150504,150505,0006641040,A2PTSM496CF40Z,"Jason A. Teeple ""Nobody made a greater mistak...",1,1,positive,1210809600,A classic,Get the movie or sound track and sing along wi...
150503,150504,0006641040,AQEYF1AXARWJZ,"Les Sinclair ""book maven""",1,1,positive,1212278400,Chicken Soup with Rice,A very entertaining rhyming story--cleaver and...
150502,150503,0006641040,A3R5XMPFU8YZ4D,"Her Royal Motherliness ""Nana""",1,1,positive,1233964800,so fun to read,This is my grand daughter's and my favorite bo...


In [102]:
final = sorteddata.drop_duplicates(subset=("UserId",'Time','Text','ProfileName'),keep='first',inplace=False)

In [103]:
final.shape

(393933, 10)

In [104]:
data.shape

(568454, 10)

In [105]:
(final['Id'].size/data['Id'].size)*100

69.29901100176971

In [106]:
final.shape

(393933, 10)

Taking subset of data from the dataset

In [107]:
final = final.sample(n=100000)
final.shape

(100000, 10)

Keeping only Reviews, Score and discarding the remaining features  

In [108]:
final = final[['Text', 'Score']]
print(final.shape)
final.head(3)

(100000, 2)


,Text,Score
460022,(Mr. Master Reviewer ^.^)<br /><br />This Ener...,positive
544096,I guess I'm not lucky the other guys.in my cas...,negative
564627,We started purchasing Eight O'clock Coffee abo...,positive


Converting all text to lower case

In [109]:
final['Text'] = final['Text'].str.lower()

In [110]:
nu_top_words = 5000

In [111]:
sentences = final['Text'].str.cat(sep = ' ')

In [112]:
words = nltk.tokenize.word_tokenize(sentences)

In [113]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to C:\Users\Anil
[nltk_data]     Chowdary\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [114]:
len(words)

9542183

In [115]:
words

['(',
 'mr.',
 'master',
 'reviewer',
 '^.^',
 ')',
 '<',
 'br',
 '/',
 '>',
 '<',
 'br',
 '/',
 '>',
 'this',
 'energy',
 'gel',
 '24',
 'pack',
 'is',
 'simply',
 'the',
 'best',
 'buy',
 'for',
 'any',
 'person',
 'who',
 'is',
 'in',
 'doing',
 'marathons',
 'or',
 'simply',
 'training',
 'the',
 'body',
 '.',
 'i',
 'have',
 'gone',
 'through',
 'at',
 'least',
 '8',
 'orders',
 'of',
 'this',
 'stuff',
 'until',
 'i',
 'finally',
 'realized',
 'how',
 'much',
 'of',
 'a',
 'killer',
 'it',
 'can',
 'be',
 'on',
 'the',
 'average',
 'user',
 "'s",
 'wallet',
 '.',
 'if',
 'these',
 'things',
 'were',
 'cheaper',
 'or',
 'if',
 'i',
 'could',
 'get',
 'them',
 'for',
 'an',
 'affordable',
 'price',
 ',',
 'they',
 'would',
 'trully',
 'be',
 'amazing',
 'but',
 'either',
 'way',
 ',',
 'this',
 'stuff',
 'helps',
 '!',
 'flavors',
 'and',
 'extremely',
 'tasty',
 'and',
 'regardless',
 'of',
 'the',
 'type',
 ',',
 'they',
 'all',
 'go',
 'down',
 'and',
 'about.',
 '<',
 'br',
 '/

In [116]:
word_dict = nltk.FreqDist(words)

In [117]:
word_dict

FreqDist({'(': 35380,
          'mr.': 114,
          'master': 85,
          'reviewer': 905,
          '^.^': 3,
          ')': 38078,
          '<': 119401,
          'br': 113681,
          '/': 114085,
          '>': 119781,
          'this': 120930,
          'energy': 2436,
          'gel': 191,
          '24': 965,
          'pack': 5145,
          'is': 132579,
          'simply': 1582,
          'the': 327317,
          'best': 13324,
          'buy': 13517,
          'for': 95763,
          'any': 11399,
          'person': 1005,
          'who': 7991,
          'in': 97932,
          'doing': 1184,
          'marathons': 9,
          'or': 31696,
          'training': 535,
          'body': 1285,
          '.': 420578,
          'i': 295160,
          'have': 61096,
          'gone': 1408,
          'through': 4418,
          'at': 31686,
          'least': 2900,
          '8': 1784,
          'orders': 285,
          'of': 138804,
          'stuff': 6603,
          'until'

In [118]:
top_words = pd.DataFrame(word_dict.most_common(nu_top_words), columns= ['words', 'frequency'])

In [119]:
top_words

,words,frequency
0,.,420578
1,the,327317
2,",",304782
3,i,295160
4,and,224509
5,a,212402
6,it,191898
7,to,177086
8,of,138804
9,is,132579


In [120]:
top_dict = {}
for idx,tup in enumerate(word_dict.most_common(nu_top_words)):
    top_dict[tup[0]] = idx

In [121]:
top_dict

{'.': 0,
 'the': 1,
 ',': 2,
 'i': 3,
 'and': 4,
 'a': 5,
 'it': 6,
 'to': 7,
 'of': 8,
 'is': 9,
 'this': 10,
 '>': 11,
 '<': 12,
 '/': 13,
 'br': 14,
 'in': 15,
 'for': 16,
 'that': 17,
 'my': 18,
 '!': 19,
 'you': 20,
 'but': 21,
 'with': 22,
 'have': 23,
 'not': 24,
 'was': 25,
 'are': 26,
 'they': 27,
 "'s": 28,
 "n't": 29,
 'as': 30,
 'on': 31,
 'so': 32,
 'like': 33,
 ')': 34,
 'these': 35,
 '(': 36,
 'good': 37,
 'them': 38,
 'be': 39,
 'or': 40,
 'at': 41,
 'just': 42,
 'taste': 43,
 'very': 44,
 'if': 45,
 'one': 46,
 'do': 47,
 'great': 48,
 'product': 49,
 'can': 50,
 'all': 51,
 'from': 52,
 "''": 53,
 'flavor': 54,
 'coffee': 55,
 'we': 56,
 'tea': 57,
 'when': 58,
 'would': 59,
 'more': 60,
 'has': 61,
 'had': 62,
 'will': 63,
 'me': 64,
 'love': 65,
 'than': 66,
 'out': 67,
 '...': 68,
 '``': 69,
 'other': 70,
 'no': 71,
 'get': 72,
 'some': 73,
 'about': 74,
 'really': 75,
 'only': 76,
 'up': 77,
 'food': 78,
 'much': 79,
 'use': 80,
 'an': 81,
 'too': 82,
 'there': 83

In [122]:
for idx,row in final.iterrows():
    words = nltk.tokenize.word_tokenize(row['Text'])
    words_list = []
    for word in words:
        if word in top_dict.keys():
            words_list.append(top_dict.get(word))
    final.set_value(idx,'Text',words_list)
            
    

C:\Users\Anil Chowdary\Anaconda3\lib\site-packages\ipykernel_launcher.py:7: FutureWarning: set_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead
  import sys


In [123]:
final.head(5)

,Text,Score
460022,"[36, 3731, 4475, 923, 34, 12, 14, 13, 11, 12, ...",positive
544096,"[3, 554, 3, 103, 24, 1993, 1, 70, 18, 363, 30,...",negative
564627,"[56, 445, 776, 1769, 3331, 55, 74, 1604, 185, ...",positive
403438,"[35, 26, 543, 4, 27, 47, 29, 23, 1, 1003, 251,...",positive
8842,"[61, 7, 39, 1, 1179, 55, 3, 87, 198, 62, 0, 6,...",negative


In [124]:
final.shape

(100000, 2)

In [159]:
final['Score'] = final['Score'].replace('positive',1)

In [160]:
final['Score'] = final['Score'].replace('negative',0)

In [161]:
final['Score']

460022    1
544096    0
564627    1
403438    1
8842      0
406370    1
552938    1
304305    1
68732     1
159646    1
115320    1
566755    1
526225    0
193379    1
385442    1
448751    1
320497    1
316795    1
134606    1
381856    1
184417    1
124358    1
550509    1
150596    1
172317    1
481750    0
150064    1
428337    1
86774     1
460397    0
         ..
57919     1
90376     1
43027     1
202070    1
305543    1
160428    1
74950     1
80385     0
369828    1
391425    1
473985    1
13008     1
71410     1
180680    1
226009    1
14135     1
97039     1
218920    1
113439    1
47763     1
404670    1
51315     1
374749    1
479805    0
196415    1
183667    0
44637     1
394493    1
445955    1
281361    1
Name: Score, Length: 100000, dtype: int64

In [162]:
x_train = final['Text'][0:70000]
x_test = final['Text'][70000:]
y_train = final['Score'][0:70000]
y_test = final['Score'][70000:]

In [163]:
print(x_train.shape)
print(x_test.shape)
print(y_train.shape)
print(y_test.shape)

(70000,)
(30000,)
(70000,)
(30000,)


In [164]:
from keras.preprocessing import sequence
from keras.layers import Embedding
from keras.models import Sequential
from keras.layers import LSTM
from keras.layers import Dense

PADDING

In [165]:
max_length = 600
x_train = sequence.pad_sequences(x_train, maxlen= max_length)
x_test = sequence.pad_sequences(x_test, maxlen= max_length)


In [166]:
print(x_train.shape)
print(x_test.shape)

(70000, 600)
(30000, 600)


1 LAYER LSTM MODEL WITH 100 LSTM CELLS

In [175]:
embed_vector_length = 32
model = Sequential()
model.add(Embedding(nu_top_words, embed_vector_length, input_length= max_length))
model.add(LSTM(100))
model.add(Dense(1,activation = 'sigmoid'))
model.compile(loss = 'binary_crossentropy', optimizer = 'adam', metrics = ['accuracy'])
print(model.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_18 (Embedding)     (None, 600, 32)           160000    
_________________________________________________________________
lstm_34 (LSTM)               (None, 100)               53200     
_________________________________________________________________
dense_14 (Dense)             (None, 1)                 101       
Total params: 213,301
Trainable params: 213,301
Non-trainable params: 0
_________________________________________________________________
None


In [135]:
model.fit(x_train, y_train, batch_size= 128, nb_epoch = 15)
scores = model.evaluate(x_test, y_test)
print("Accuracy", scores[1]*100)

C:\Users\Anil Chowdary\Anaconda3\lib\site-packages\keras\models.py:981: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  warnings.warn('The `nb_epoch` argument in `fit` '


Epoch 1/15
70000/70000 [==============================] - 820s 12ms/step - loss: 0.2676 - acc: 0.8910
Epoch 2/15
70000/70000 [==============================] - 787s 11ms/step - loss: 0.2455 - acc: 0.9005
Epoch 3/15
70000/70000 [==============================] - 699s 10ms/step - loss: 0.2307 - acc: 0.9083
Epoch 4/15
70000/70000 [==============================] - 703s 10ms/step - loss: 0.2172 - acc: 0.9130
Epoch 5/15
70000/70000 [==============================] - 700s 10ms/step - loss: 0.2011 - acc: 0.9210
Epoch 6/15
70000/70000 [==============================] - 698s 10ms/step - loss: 0.1816 - acc: 0.9291
Epoch 7/15
70000/70000 [==============================] - 696s 10ms/step - loss: 0.1670 - acc: 0.9356
Epoch 8/15
70000/70000 [==============================] - 701s 10ms/step - loss: 0.1557 - acc: 0.9418
Epoch 9/15
70000/70000 [==============================] - 728s 10ms/step - loss: 0.1434 - acc: 0.9471
Epoch 10/15
70000/70000 [==============================] - 771s 11ms/step - loss: 

3 LAYER LSTM MODEL WITH 100, 50 AND 25 CELLS IN THE LAYERS

In [174]:
embed_vector_length = 32
model = Sequential()
model.add(Embedding(nu_top_words, embed_vector_length, input_length= max_length))
model.add(LSTM(100, return_sequences= True))
model.add(LSTM(50, return_sequences= True))
model.add(LSTM(25, return_sequences= True))
model.add(Dense(1,activation = 'sigmoid'))
model.compile(loss = 'binary_crossentropy', optimizer = 'adam', metrics = ['accuracy'])
print(model.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_17 (Embedding)     (None, 600, 32)           160000    
_________________________________________________________________
lstm_31 (LSTM)               (None, 600, 100)          53200     
_________________________________________________________________
lstm_32 (LSTM)               (None, 600, 50)           30200     
_________________________________________________________________
lstm_33 (LSTM)               (None, 600, 25)           7600      
_________________________________________________________________
dense_13 (Dense)             (None, 600, 1)            26        
Total params: 251,026
Trainable params: 251,026
Non-trainable params: 0
_________________________________________________________________
None


In [176]:
model.fit(x_train, y_train, batch_size= 128, nb_epoch = 15)
scores = model.evaluate(x_test, y_test)
print("Accuracy", scores[1]*100)

C:\Users\Anil Chowdary\Anaconda3\lib\site-packages\keras\models.py:981: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  warnings.warn('The `nb_epoch` argument in `fit` '


Epoch 1/15
70000/70000 [==============================] - 779s 11ms/step - loss: 0.3687 - acc: 0.8380
Epoch 2/15
70000/70000 [==============================] - 917s 13ms/step - loss: 0.2726 - acc: 0.8874
Epoch 3/15
70000/70000 [==============================] - 816s 12ms/step - loss: 0.2492 - acc: 0.8982
Epoch 4/15
70000/70000 [==============================] - 944s 13ms/step - loss: 0.2327 - acc: 0.9057
Epoch 5/15
70000/70000 [==============================] - 744s 11ms/step - loss: 0.2155 - acc: 0.9128
Epoch 6/15
70000/70000 [==============================] - 710s 10ms/step - loss: 0.2034 - acc: 0.9192
Epoch 7/15
70000/70000 [==============================] - 798s 11ms/step - loss: 0.1835 - acc: 0.9282
Epoch 8/15
70000/70000 [==============================] - 744s 11ms/step - loss: 0.1706 - acc: 0.9350
Epoch 9/15
70000/70000 [==============================] - 813s 12ms/step - loss: 0.1574 - acc: 0.9404
Epoch 10/15
70000/70000 [==============================] - 685s 10ms/step - loss: 

CONCLUSION:
    
    -> The total length of the corpus is 9542183
    
    -> After removing all the duplicates and cleaning all the sentences and here we considered the top 5000 words
    
    -> Frequency of each word is calculated and indexed them based on the frequency
    
    -> All the words in the sentences are replaced by the index positions of the top 5000 words
    
    -> If a word in sentence is not present in top 5000 then it is discarded
    
    -> Now we have all the reviews with the words replaced by their index positions in the frequency distribution
    
    -> Padding all the sentences to have same length to perform batch operations
    
    At the end of 15 epochs:
        
        Single LSTM Layer with 100 cells:
            
                    Train loss: 0.0946        Accuracy: 96.80
                        
                    Accuracy on test data is 89.15
                    
        3 Layer LSTM model with 100, 50 and 25 cells in the consecutive layers:
            
                    Train loss: 0.0961       Accuracy: 96.62
                        
                    Accuracy on the test data is 88.87